In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import LeaguesModel

leagues_model = LeaguesModel()

leagues_model.load_counties_geojson()

In [ ]:
leagues_model.load_counties_data()

### Blank Sandbox

In [ ]:
leagues_model.reset_county_styles()

leagues_model.render_map(None)

In [ ]:
from sportwarz import LeagueJson

import json
import codecs
import ipywidgets as widgets

leagues = { }

league_input = widgets.Dropdown(description = "League:")
add_button = widgets.Button(description="Add Team")

error_message = widgets.Label(style = { "color" : "red" })

out = widgets.Output()
out.clear_output()

load_league_input = widgets.Dropdown(description = "League to Load:",
                                      options = ["NFL", "MLB", "NBA", "NHL", "MLS"])
load_league_button = widgets.Button(description="Load League")

upload = widgets.FileUpload(
    accept='.json', 
    multiple= False  
)

add_team_league_input = widgets.Dropdown()
add_team_textarea = widgets.Textarea(value = """[{
    "name": "Wichita Whatevers",
    "venue": "Venue Arena Stadium",
      "L": 1.0,
      "S": 3.0,
      "N": 3.0,
      "coordinates": {
        "lat":  37.7,
        "lon": -97.3
      },
      "color": "#001122",
      "state": "Kansas"
    }                                     
]""", layout=widgets.Layout(width="400px", height="300px"))
add_team_button = widgets.Button(description = "Add/Update Team(s)")

delete_league_input = widgets.Dropdown(description = "League:")
delete_team_input = widgets.Dropdown(description = "Team:")

delete_button = widgets.Button(description="Delete Team")

load_team_panel = widgets.VBox([
    load_league_input,
    load_league_button
])

upload_panel = widgets.VBox([
    upload,    
])

add_panel = widgets.VBox([
    add_team_league_input,  
    add_team_textarea, 
    add_team_button 
], layout=widgets.Layout(width="100%"))

delete_panel = widgets.VBox([
    delete_league_input,  
    delete_team_input, 
    delete_button 
])

tab = widgets.Tab()
tab.children = [load_team_panel, upload_panel, add_panel, delete_panel]
tab.titles =   ["Load League", "Upload League (JSON)", "Add/Update Teams (JSON)", "Delete Team"]

league_dropdown_options = []

@out.capture()
def lock_tabs(locked=True):
    for child in tab.children:
        for widget in child.children:
            if hasattr(widget, "disabled"):
                widget.disabled = locked

@out.capture()
def add_new_teams(button):
    lock_tabs(True)
    error_message.value = ""
    try:
        if add_team_league_input.value:
            new_teams = json.loads(add_team_textarea.value)
            leagues_model.add_teams(add_team_league_input.value, new_teams)
            leagues_model.calculate_distances() 
            leagues_model.compute_shares()  
            leagues_model.compute_output_dataframe() 
    
            leagues_model.heatmap_counties(add_team_league_input.value)
            leagues_model.refresh_geojson_layer()
        else:
            error_message.value = "League not selected"
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False)

@out.capture()
def delete_team(button):
    lock_tabs(True)
    error_message.value = ""
    try:
        if delete_league_input.value:
            if leagues_model.delete_teams(delete_league_input.value, [delete_team_input.value]):
                leagues_model.calculate_distances() 
                leagues_model.compute_shares()  
                leagues_model.compute_output_dataframe() 
    
                leagues_model.heatmap_counties(delete_league_input.value)
                leagues_model.refresh_geojson_layer()
        else:
            error_message.value = "League not selected"
    finally:
        lock_tabs(False)

@out.capture()
def delete_league_change(change):
    delete_team_input.options = list(map(lambda team: team["name"], leagues_model._leagues[delete_league_input.value]["teams"]))

@out.capture()
def load_league(button):
    lock_tabs(True)
    error_message.value = ""
    try:
        league_name = load_league_input.value 
        if not league_name in leagues_model._leagues:
            leagues_model.load_leagues([league_name])
            leagues_model.calculate_distances() 
            leagues_model.compute_shares()  
            leagues_model.compute_output_dataframe() 
    
        leagues_model.heatmap_counties(league_name)
        leagues_model.refresh_geojson_layer()
      
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)  
            delete_league_input.options =league_dropdown_options
            add_team_league_input.options = league_dropdown_options  
    finally:
        lock_tabs(False)


@out.capture()
def on_upload_change(change):
    lock_tabs(True)
    error_message.value = ""
    try:
        file_info = change["new"][0]
        content = codecs.decode(file_info.content, encoding="utf-8")
        league_json: LeagueJson = json.loads(content)
        league_name = league_json["league_name"]
        leagues_model.add_league(league_json)
        leagues_model.calculate_distances() 
        leagues_model.compute_shares()  
        leagues_model.compute_output_dataframe() 
    
        leagues_model.heatmap_counties(league_name)
        leagues_model.refresh_geojson_layer()
      
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)  
            delete_league_input.options =league_dropdown_options
            add_team_league_input.options = league_dropdown_options        
    finally:
        lock_tabs(False)

load_league_button.on_click(load_league)

upload.observe(on_upload_change, names="value")  

add_team_button.on_click(add_new_teams) 

delete_league_input.observe(delete_league_change, names='value')
delete_button.on_click(delete_team) 

widgets.VBox([
    tab,
    error_message
])


In [ ]:
out